In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
import json
import requests

# Load environment variables from the .env file
load_dotenv()

# Database file path
database_file_path = "./db/test.db"

# Create the SQLAlchemy engine
engine = create_engine(f'sqlite:///{database_file_path}')

@tool
def calculate_price(product_list: list) -> float:
    """
    Calculates the total price of a list of products.

    Args:
        product_list (list): A list of dictionaries representing products.
        
    Returns:
        float: The total price of the products.
    """
    total_price = sum(product['price'] for product in product_list)
    return total_price

@tool
def get_product_price(product_name: str) -> str:
    """
    Retrieves the price of a specific product.

    Args:
        product_name (str): The name of the product.
        
    Returns:
        str: The price of the product in JSON format.
    """
    try:
        query = text("""
            SELECT name, price
            FROM Product
            WHERE name LIKE :product_name;
        """)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection, params={"product_name": f"%{product_name}%"})
        
        if not result.empty:
            return json.dumps(result.to_dict('records')[0])
        else:
            return json.dumps({"product_name": product_name, "price": None})
    except Exception as e:
        return json.dumps({"error": str(e)})

@tool
def get_products_available() -> str:
    """
    Retrieves the list of products that are available.

    Returns:
        str: A list of available products in JSON format.
    """
    try: 
        query = text("""
            SELECT name, price
            FROM Product
            WHERE available = 1;
        """)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        
        if not result.empty:
            return json.dumps(result.to_dict('records'))
        else:
            return json.dumps({"available_products": []})
    except Exception as e:
        return json.dumps({"error": str(e)})

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Define the tools
tools_sql = [get_product_price, get_products_available, calculate_price]

# Get the API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Create the agent
agent = create_tool_calling_agent(llm, tools_sql, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_sql, verbose=True)

# Example usage
# agent_executor.invoke({"input": "What is the total cost for a portable charger and a keyboard?"})


/Users/juliomiranda/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Tools / API calls

In [51]:
baseURL = 'https://dashboard.colorines.paitesting.com/api'
base_url = 'https://dashboard.colorines.paitesting.com/api'

### Token

In [52]:
# Set the URL for the token API
url_token = baseURL+"/v1/token"

# Set the data for the POST request
data = {
    "email": "sale@platform.com",
    "password": os.getenv('CONSULTANT_PASSWORD'),
    "device_name": "colorina_2",
    "role": "consultant"
}

# Send a POST request
response = requests.post(url_token, json=data)

print(response)

# Extract the token from the response
token = response.json().get('data').get('token')

print("Token:", token)


<Response [200]>
Token: 1996|RPemlkBYvOcm8TXg4ThPXWlSKlkRkXkGgSUSxdcK400494d6


In [61]:
from langchain_core.tools import tool
import requests
import json

@tool
def get_sales(token: str) -> str:
    """
    Uses the token to fetch a list of sales from colorines API.
    
    Args:
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    headers = {"Authorization": f"Bearer {token}"}
    url= "https://dashboard.colorines.paitesting.com/api/v1/sales"
    try:
        response = requests.get(url, headers=headers)
        # Check if the response was successful
        if response.status_code == 200:
            try:
                return json.dumps(response.json())  # Convert JSON response to string
            except ValueError:  # Catch JSON errors
                return json.dumps({"error": "Error parsing JSON"})
        else:
            return json.dumps({"error": f"Received status code {response.status_code}"})
    except requests.exceptions.RequestException as e:
        return json.dumps({"error": str(e)})
    
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Example of integrating with langchain agent framework
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Add the `get_sales` function to the list of tools
tools_sql.append(get_sales)  # Assuming 'tools_sql' already exists and includes other tools

# Create the agent
agent = create_tool_calling_agent(llm, tools_sql, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools_sql, verbose=True)

# Example usage:
# token = 'your_actual_token_here'
# base_url = 'https://api.yourdomain.com'



In [60]:
from langchain_core.tools import BaseTool
import requests

class APITool(BaseTool):
    name = "fetch_api_data"
    description = "Fetch data from the API with authentication."
    
    def __init__(self, api_token):
        super().__init__()
        self.api_token = api_token

    def _run(self, query: str) -> str:
        headers = {"Authorization": f"Bearer {self.api_token}"}
        url = "https://dashboard.colorines.paitesting.com/api/v1/sales"
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.json()
            else:
                return {"error": f"Received status code {response.status_code}"}
        except requests.exceptions.RequestException as e:
            return {"error": str(e)}

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Example of integrating with langchain agent framework
api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)

# Define the token
token = "1911|VhdgL4Sr8hE8DKvIpJupBWTkdYSS1g7WwxJcNkP5650a0619"

# Create an instance of the APITool with the authentication token
api_tool = APITool(api_token=token)

# Add the tool to the list of tools
tools = [api_tool]

# Create the agent with the tools
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Example usage:
result = agent_executor.run("Fetch data from the API")
print(result)


ValueError: "APITool" object has no field "api_token"

In [55]:
token

'1996|RPemlkBYvOcm8TXg4ThPXWlSKlkRkXkGgSUSxdcK400494d6'

In [56]:
input_data = {
    "input": "Show me info about my last sale",
    "token": token
}

# Invoke the agent
agent_executor.invoke(input_data)



> Entering new AgentExecutor chain...

Invoking: `get_sales` with `{'token': 'abc123'}`


{"error": "Error parsing JSON"}I encountered an error while trying to fetch information about your last sale. Please try again later.

> Finished chain.


{'input': 'Show me info about my last sale',
 'token': '1996|RPemlkBYvOcm8TXg4ThPXWlSKlkRkXkGgSUSxdcK400494d6',
 'output': 'I encountered an error while trying to fetch information about your last sale. Please try again later.'}

## Sales

In [ ]:
def get_sales(base_url: str, token: str) -> str:
    """
    Fetches a list of sales from colorines API.
    
    Args:
        base_url (str): The base URL of the API.
        token (str): The authorization token required to access the API.

    Returns:
        str: The JSON response from the API as a string.
    """
    headers = {"Authorization": f"Bearer {token}"}
    url_list_sales = f"{base_url}/v1/sales"
    try:
        response = requests.get(url_list_sales, headers=headers)
        # Check if the response was successful
        if response.status_code == 200:
            try:
                return json.dumps(response.json())  # Convert JSON response to string
            except ValueError:  # Catch JSON errors
                return json.dumps({"error": "Error parsing JSON"})
        else:
            return json.dumps({"error": f"Received status code {response.status_code}"})
    except requests.exceptions.RequestException as e:
        return json.dumps({"error": str(e)})
    
sales_data = get_sales(base_url, token)
sales_data

In [ ]:

def get_sales(base_url: str, token: str):
    """
    Fetches a list of sales from the specified API endpoint.

    Args:
    - base_url (str): The base URL of the API.
    - token (str): The authorization token required to access the API.

    Returns:
    - str: The JSON response from the API as a string.
    """
    headers = {"Authorization": f"Bearer {token}"}
    url_list_sales = f"{base_url}/v1/sales"
    response = requests.get(url_list_sales, headers=headers)
    print(response)
    # Check if the response was successful
    if response.status_code == 200:
        try:
            return response.json()  # Try to parse JSON if the status code is 200
        except ValueError:  # Catch JSON errors
            return "Error parsing JSON"
    else:
        return f"Error: Received status code {response.status_code}"

# Example usage:
# token = 'your_actual_token_here'
# base_url = 'https://api.yourdomain.com'
sales_data = get_sales(base_url, token)
sales_data

In [ ]:
@tool
def get_customers(base_url: str, token: str) -> str:
    """
    Fetches a list of customers from Colorines API endpoint.

    Args:
    - base_url (str): The base URL of the API.
    - token (str): The authorization token required to access the API.

    Returns:
    - str: The JSON response from the API as a string.
    """
    headers = {
        "Authorization": f"Bearer {token}"
    }
    url_list_customers = f"{base_url}/v1/customers"
    response = requests.get(url_list_customers, headers=headers)
    return response.json()

# Example usage:
customers_data = get_customers(base_url, token)
customers_data

In [3]:
# Prueba la ejecución del agente
# agent_executor.invoke({"input": "Quiero comprar Un teclado un una camara cuanto me costaria"})

# Prueba de disponibilidad de productos
# agent_executor.invoke({"input": "Que productos tienes disponibles?"})
agent_executor.invoke({"input": "Quiero comprar un Cargador portátil y un teclado, cual es el total?"})



> Entering new AgentExecutor chain...

Invoking: `get_products_available` with `{}`


[{"name": "Teclado", "price": 75.25}, {"name": "Laptop", "price": 1000.0}, {"name": "Smartphone", "price": 800.0}, {"name": "Tablet", "price": 300.0}, {"name": "Impresora", "price": 200.0}, {"name": "Mano", "price": 150.0}, {"name": "C\u00e1mara", "price": 400.0}, {"name": "Televisor", "price": 700.0}, {"name": "Router", "price": 80.0}, {"name": "Reproductor Blu-ray", "price": 180.0}, {"name": "Teclado inal\u00e1mbrico", "price": 60.0}, {"name": "Mouse inal\u00e1mbrico", "price": 80.0}, {"name": "Webcam", "price": 70.0}, {"name": "Tarjeta de video", "price": 250.0}, {"name": "Memoria RAM", "price": 120.0}, {"name": "Disco duro externo", "price": 150.0}, {"name": "Tarjeta madre", "price": 350.0}, {"name": "Procesador", "price": 300.0}, {"name": "Gabinete para PC", "price": 80.0}, {"name": "Fuente de poder", "price": 100.0}, {"name": "Router inal\u00e1mbrico", "price": 50.0}, {"name": "Adaptador Wi-Fi

{'input': 'Quiero comprar un Cargador portátil y un teclado, cual es el total?',
 'output': 'El total para comprar un Cargador portátil y un teclado es de $115.25.'}